# Setup the database with docker

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text, Table, Column, Integer, String, Float, DateTime, MetaData
from datetime import datetime
import warnings
import math
from urllib.parse import quote_plus

warnings.simplefilter("ignore")

print("Starting Master ETL Pipeline...")

DB_USER = 'admin'
DB_PASS = 'Admin@1234Strong!'  
DB_HOST = '127.0.0.1'
DB_PORT = '3306'
TARGET_DB = 'lending_club'

CSV_PATH = "/home/nanak/mlops/data/raw/loan.csv"

DB_PASS_ENCODED = quote_plus(DB_PASS)


def safe_float(val, default=0.0):
    try:
        if pd.isna(val) or val is None: return default
        f = float(val)
        return default if math.isnan(f) else f
    except: return default

def parse_percent(val):
    try:
        if isinstance(val, str): return safe_float(val.strip('%'))
        return safe_float(val)
    except: return 0.0

def parse_term(val):
    try: return safe_float(str(val).strip().split()[0])
    except: return None

def parse_date(val):
    try: return datetime.strptime(str(val), '%b-%Y')
    except: return None

def map_emp(val):
    m = {'< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
         '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10}
    return m.get(val, 0)

def calculate_fico_score(low, high):
    """Calculates average FICO and ensures it is within logical bounds."""
    if low is None or high is None:
        return 600.0 # Default fallback
    score = (safe_float(low) + safe_float(high)) / 2
    return max(300, min(850, score))


print("\n Establishing Database Connection...")

server_url = f"mysql+pymysql://{DB_USER}:{DB_PASS_ENCODED}@{DB_HOST}:{DB_PORT}/"
engine_server = create_engine(server_url)

try:
    with engine_server.connect() as conn:
        conn.execution_options(isolation_level="AUTOCOMMIT")
        conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {TARGET_DB};"))
    print(f"Database '{TARGET_DB}' verified/created.")
except Exception as e:
    print(f"Connection Error: {e}")
    raise e

engine = create_engine(f"{server_url}{TARGET_DB}")
metadata = MetaData()

loans_clean = Table(
    'loans_clean', metadata,
    Column('target', Integer),
    Column('loan_amnt', Float),
    Column('int_rate', Float),
    Column('term', Float),
    Column('grade', String(5)),
    Column('sub_grade', String(5)), 
    Column('purpose', String(100)),
    Column('application_type', String(50)), 
    Column('annual_inc', Float),
    Column('dti', Float),
    Column('emp_length_num', Float),
    Column('home_ownership', String(50)),
    Column('verification_status', Integer),
    Column('fico_score', Float),
    Column('revol_util', Float),
    Column('open_acc', Float),
    Column('pub_rec', Float),
    Column('addr_state', String(5)),
    Column('issue_date', DateTime),
    Column('issue_year', Integer)
)

metadata.drop_all(engine)
metadata.create_all(engine)


print("\n Running ETL (Extract -> Transform -> Load)...")

chunk_size = 10000
total_rows = 0

try:
    print(f"   -> Processing CSV Stream: {CSV_PATH}")g
    for chunk in pd.read_csv(CSV_PATH, chunksize=chunk_size, low_memory=False):
        
        chunk = chunk[chunk['loan_status'].isin(['Fully Paid', 'Charged Off'])].copy()
        if chunk.empty: continue
        
        chunk.columns = [c.replace(' ', '_').lower() for c in chunk.columns]
        
        batch_data = []
        for _, row in chunk.iterrows():
            dt = parse_date(row.get('issue_d'))
            
            clean_row = {
                'target': 1 if row['loan_status'] == 'Charged Off' else 0,
                'loan_amnt': safe_float(row.get('loan_amnt')),
                'int_rate': safe_float(row.get('int_rate')),
                'term': parse_term(row.get('term')),
                'grade': str(row.get('grade')),
                'sub_grade': str(row.get('sub_grade')),
                'purpose': str(row.get('purpose')),
                'application_type': str(row.get('application_type')),
                'annual_inc': safe_float(row.get('annual_inc')),
                'dti': safe_float(row.get('dti')),
                'emp_length_num': map_emp(row.get('emp_length')),
                'home_ownership': str(row.get('home_ownership')),
                'verification_status': 0 if row.get('verification_status') == 'Not Verified' else 1,
                'fico_score': calculate_fico_score(row.get('fico_range_low'), row.get('fico_range_high')),
                'revol_util': parse_percent(row.get('revol_util')),
                'open_acc': safe_float(row.get('open_acc')),
                'pub_rec': safe_float(row.get('pub_rec')),
                'addr_state': str(row.get('addr_state')),
                'issue_date': dt,
                'issue_year': dt.year if dt else None
            }
            batch_data.append(clean_row)

        if batch_data:
            with engine.begin() as conn:
                conn.execute(loans_clean.insert(), batch_data)
            total_rows += len(batch_data)
            print(f"      -> Inserted {total_rows} rows...")

    print("ETL SUCCESS: MariaDB table 'loans_clean' is fully populated.")

except Exception as e:
    print(f"ETL Failed: {e}")
    raise e


print("Validating Final SQL Data...")

try:
    with engine.connect() as conn:
        # 1. Target check
        target_check = conn.execute(text("SELECT COUNT(*) FROM loans_clean WHERE target NOT IN (0, 1)")).scalar()
        # 2. FICO check
        fico_check = conn.execute(text("SELECT COUNT(*) FROM loans_clean WHERE fico_score < 300 OR fico_score > 850")).scalar()
        # 3. Income check
        income_check = conn.execute(text("SELECT COUNT(*) FROM loans_clean WHERE annual_inc IS NULL")).scalar()

    if target_check == 0 and fico_check == 0 and income_check == 0:
        print("FINAL SUCCESS: Pipeline Verified! Data is ready for Feature Engineering.")
    else:
        print(f"WARNING: Data issues found! Targets: {target_check}, FICO errors: {fico_check}, NULL Incomes: {income_check}")

except Exception as e:
    print(f" Error during Post-Validation: {e}")

🚀 Starting Master ETL Pipeline...

🔌 [Phase 1] Establishing Database Connection...
✅ Database 'lending_club' verified/created.
⚙️ Recreating table schema...

⚙️ [Phase 2] Running ETL (Extract -> Transform -> Load)...
   -> Processing CSV Stream: /home/nanak/mlops/data/raw/loan.csv
      -> Inserted 178 rows...
      -> Inserted 388 rows...
      -> Inserted 606 rows...
      -> Inserted 823 rows...
      -> Inserted 1039 rows...
      -> Inserted 1294 rows...
      -> Inserted 1593 rows...
      -> Inserted 1855 rows...
      -> Inserted 2163 rows...
      -> Inserted 2450 rows...
      -> Inserted 2793 rows...
      -> Inserted 3162 rows...
      -> Inserted 3527 rows...
      -> Inserted 3948 rows...
      -> Inserted 4394 rows...
      -> Inserted 4908 rows...
      -> Inserted 5415 rows...
      -> Inserted 5984 rows...
      -> Inserted 6610 rows...
      -> Inserted 7186 rows...
      -> Inserted 7840 rows...
      -> Inserted 8523 rows...
      -> Inserted 9299 rows...
      -> 

In [ ]:
# re ETL

In [1]:
# import pandas as pd
# import numpy as np
# from sqlalchemy import create_engine, text, Table, Column, Integer, String, Float, DateTime, MetaData
# from datetime import datetime
# import warnings
# import math
# from urllib.parse import quote_plus
# import csv

# # 1. Hide warnings
# warnings.simplefilter("ignore")

# print("🚀 Starting Advanced Model-Ready ETL Pipeline...")

# # ==========================================
# # PART 1: CONFIGURATION
# # ==========================================
# DB_USER, DB_PASS = 'admin', 'Admin@1234Strong!'
# DB_HOST, DB_PORT = '127.0.0.1', '3306'
# TARGET_DB = 'lending_club'
# CSV_PATH = "/home/nanak/mlops/data/raw/loan.csv"

# DB_PASS_ENCODED = quote_plus(DB_PASS)

# # --- Enhanced Helper Functions ---
# def safe_float(val, default=0.0):
#     try:
#         if val is None or str(val).strip() == "" or str(val).lower() == 'nan': 
#             return default
#         return float(str(val).replace('%', '').strip())
#     except: return default

# def map_emp_length(val):
#     m = {'< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
#          '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10}
#     return float(m.get(val, 0))

# # ==========================================
# # PART 2: DATABASE INITIALIZATION
# # ==========================================
# server_url = f"mysql+pymysql://{DB_USER}:{DB_PASS_ENCODED}@{DB_HOST}:{DB_PORT}/"
# engine = create_engine(f"{server_url}{TARGET_DB}")
# metadata = MetaData()

# # Define the "High Quality" Table Schema
# loans_clean = Table(
#     'loans_clean', metadata,
#     Column('target', Integer),
#     Column('loan_amnt', Float),
#     Column('int_rate', Float),
#     Column('annual_inc', Float),
#     Column('dti', Float),
#     Column('fico_score', Float), # We will use fallback logic for this
#     Column('term_months', Integer),
#     Column('emp_length_num', Float),
#     Column('delinq_2yrs', Float),
#     Column('mths_since_last_delinq', Float),
#     Column('tot_cur_bal', Float),
#     Column('revol_util', Float),
#     Column('num_tl_op_past_12m', Float),
#     Column('tot_coll_amt', Float),
#     Column('is_verified', Integer),
#     Column('grade', String(5)),
#     Column('home_ownership', String(20)),
#     Column('purpose', String(50))
# )

# print("⚙️ Dropping and recreating table...")
# metadata.drop_all(engine)
# metadata.create_all(engine)

# # ==========================================
# # PART 3: EXTRACTION & TRANSFORMATION (Streaming)
# # ==========================================
# print("\n🔄 Starting Stream Transformation...")

# total_rows = 0
# null_stats = {"fico": 0, "income": 0, "dti": 0}

# with open(CSV_PATH, 'r', encoding='utf-8') as f:
#     reader = csv.DictReader(f)
    
#     # Verification: Check if FICO columns actually exist in the header
#     headers = reader.fieldnames
#     has_fico = 'fico_range_low' in headers
    
#     batch = []
#     for row in reader:
#         # 1. FILTER: Target Definition
#         if row['loan_status'] not in ['Fully Paid', 'Charged Off']:
#             continue

#         # 2. TRANSFORM: FICO Score Logic (The "Problem Solver")
#         # If columns missing or 0, default to 680 (Neutral)
#         if has_fico:
#             f_low = safe_float(row.get('fico_range_low'))
#             f_high = safe_float(row.get('fico_range_high'))
#             fico_val = (f_low + f_high) / 2
#         else:
#             fico_val = 0
            
#         if fico_val < 300: # Invalid or Missing
#             fico_val = 680.0
#             null_stats["fico"] += 1

#         # 3. TRANSFORM: Income Imputation
#         income = safe_float(row.get('annual_inc'))
#         if income <= 0:
#             income = 50000.0 # Standard median fallback
#             null_stats["income"] += 1

#         # 4. TRANSFORM: Term Parsing
#         term_str = row.get('term', '36')
#         term_val = 36 if '36' in term_str else 60

#         # 5. MAPPING: Categoricals and Null-Heavy Columns
#         clean_row = {
#             'target': 1 if row['loan_status'] == 'Charged Off' else 0,
#             'loan_amnt': safe_float(row.get('loan_amnt')),
#             'int_rate': safe_float(row.get('int_rate')),
#             'annual_inc': income,
#             'dti': safe_float(row.get('dti')),
#             'fico_score': fico_val,
#             'term_months': term_val,
#             'emp_length_num': map_emp_length(row.get('emp_length')),
#             'delinq_2yrs': safe_float(row.get('delinq_2yrs')),
#             # Logical Null: If missing, they probably never had a delinquency. Set to 999 months.
#             'mths_since_last_delinq': safe_float(row.get('mths_since_last_delinq'), default=999.0),
#             'tot_cur_bal': safe_float(row.get('tot_cur_bal')),
#             'revol_util': safe_float(row.get('revol_util')),
#             'num_tl_op_past_12m': safe_float(row.get('num_tl_op_past_12m')),
#             'tot_coll_amt': safe_float(row.get('tot_coll_amt')),
#             'is_verified': 1 if row.get('verification_status') != 'Not Verified' else 0,
#             'grade': row.get('grade', 'U'),
#             'home_ownership': row.get('home_ownership', 'OTHER'),
#             'purpose': row.get('purpose', 'other')
#         }
        
#         batch.append(clean_row)
        
#         # 6. LOAD: Batching for Speed
#         if len(batch) >= 10000:
#             with engine.begin() as conn:
#                 conn.execute(loans_clean.insert(), batch)
#             total_rows += len(batch)
#             print(f"   -> Processed {total_rows} rows...")
#             batch = []

#     # Final batch
#     if batch:
#         with engine.begin() as conn:
#             conn.execute(loans_clean.insert(), batch)
#         total_rows += len(batch)

# print(f"\n✅ SUCCESS: Integrated {total_rows} rows.")
# print(f"📊 Quality Note: Imputed {null_stats['fico']} missing FICO scores and {null_stats['income']} missing incomes.")

In [4]:
# import csv
# import math
# import warnings
# from datetime import datetime
# from sqlalchemy import create_engine, text, Table, Column, Integer, String, Float, MetaData
# from urllib.parse import quote_plus

# warnings.simplefilter("ignore")

# # 1. Configuration
# DB_USER, DB_PASS = 'admin', 'Admin@1234Strong!'
# DB_HOST, DB_PORT = '127.0.0.1', '3306'
# TARGET_DB = 'lending_club'
# CSV_PATH = "/home/nanak/mlops/data/raw/loan.csv"

# # Connection
# PASS_ENC = quote_plus(DB_PASS)
# BASE_URL = f"mysql+pymysql://{DB_USER}:{PASS_ENC}@{DB_HOST}:{DB_PORT}/"
# engine = create_engine(f"{BASE_URL}{TARGET_DB}")

# # Helpers
# def safe_float(val, default=0.0):
#     try: return float(str(val).replace('%', '').strip()) if val else default
#     except: return default

# # ==========================================
# # 1. SCHEMA SETUP
# # ==========================================
# print("🚀 Initializing MariaDB Schema (Excluding FICO)...")
# root_engine = create_engine(BASE_URL)
# with root_engine.connect() as conn:
#     conn.execution_options(isolation_level="AUTOCOMMIT")
#     conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {TARGET_DB}"))

# metadata = MetaData()
# loans_clean = Table(
#     'loans_clean', metadata,
#     Column('target', Integer),
#     Column('loan_amnt', Float),
#     Column('int_rate', Float),
#     Column('annual_inc', Float),
#     Column('dti', Float),
#     Column('revol_util', Float),
#     Column('delinq_2yrs', Float),
#     Column('grade', String(10)),
#     Column('home_ownership', String(20)),
#     Column('term', String(20))
# )
# metadata.drop_all(engine)
# metadata.create_all(engine)

# # ==========================================
# # 2. STREAMING INGESTION
# # ==========================================
# print("⚙️ Streaming Data to DB...")
# batch = []
# with open(CSV_PATH, 'r', encoding='utf-8') as f:
#     reader = csv.DictReader(f)
#     for row in reader:
#         if row['loan_status'] not in ['Fully Paid', 'Charged Off']:
#             continue
        
#         batch.append({
#             'target': 1 if row['loan_status'] == 'Charged Off' else 0,
#             'loan_amnt': safe_float(row.get('loan_amnt')),
#             'int_rate': safe_float(row.get('int_rate')),
#             'annual_inc': safe_float(row.get('annual_inc'), default=50000.0), # Impute median
#             'dti': safe_float(row.get('dti')),
#             'revol_util': safe_float(row.get('revol_util')),
#             'delinq_2yrs': safe_float(row.get('delinq_2yrs')),
#             'grade': row.get('grade', 'U'),
#             'home_ownership': row.get('home_ownership', 'OTHER'),
#             'term': row.get('term', '36 months')
#         })
        
#         if len(batch) >= 10000:
#             with engine.begin() as conn:
#                 conn.execute(loans_clean.insert(), batch)
#             batch = []

# if batch:
#     with engine.begin() as conn:
#         conn.execute(loans_clean.insert(), batch)

# print("✅ Ingestion Complete.")

🚀 Initializing MariaDB Schema (Excluding FICO)...
⚙️ Streaming Data to DB...
✅ Ingestion Complete.


In [7]:
# columns_to_check = [
#     'loan_amnt', 'int_rate', 'annual_inc', 'dti', 'revol_util', 'delinq_2yrs', 
#     'grade', 'home_ownership', 'term'
# ]

# with engine.connect() as conn:
#     query = "SELECT " + ", ".join(
#         [f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS {col}_null" for col in columns_to_check]
#     ) + " FROM loans_clean"
    
#     result = conn.execute(text(query)).fetchone()
#     print("Missing values per column:", dict(result))


Missing values per column: {'loan_amnt_null': Decimal('0'), 'int_rate_null': Decimal('0'), 'annual_inc_null': Decimal('0'), 'dti_null': Decimal('0'), 'revol_util_null': Decimal('0'), 'delinq_2yrs_null': Decimal('0'), 'grade_null': Decimal('0'), 'home_ownership_null': Decimal('0'), 'term_null': Decimal('0')}


In [8]:
# numeric_cols = ['loan_amnt','int_rate','annual_inc','dti','revol_util','delinq_2yrs']

# with engine.connect() as conn:
#     query = "SELECT " + ", ".join(
#         [f"MIN({col}) AS min_{col}, MAX({col}) AS max_{col}" for col in numeric_cols]
#     ) + " FROM loans_clean"
    
#     result = conn.execute(text(query)).fetchone()
#     print("Numeric ranges:", dict(result))


Numeric ranges: {'min_loan_amnt': 500.0, 'max_loan_amnt': 40000.0, 'min_int_rate': 5.31, 'max_int_rate': 30.99, 'min_annual_inc': 0.0, 'max_annual_inc': 10999200.0, 'min_dti': -1.0, 'max_dti': 999.0, 'min_revol_util': 0.0, 'max_revol_util': 892.3, 'min_delinq_2yrs': 0.0, 'max_delinq_2yrs': 39.0}


In [9]:
# categorical_cols = ['grade','home_ownership','term']

# with engine.connect() as conn:
#     for col in categorical_cols:
#         print(f"\nUnique values in {col}:")
#         rows = conn.execute(text(f"SELECT {col}, COUNT(*) FROM loans_clean GROUP BY {col}")).fetchall()
#         for val, cnt in rows:
#             print(f"  {val}: {cnt}")



Unique values in grade:
  A: 226243
  B: 380152
  C: 369928
  D: 195280
  E: 91570
  F: 31483
  G: 8951

Unique values in home_ownership:
  ANY: 267
  MORTGAGE: 645496
  NONE: 48
  OTHER: 144
  OWN: 139844
  RENT: 517808

Unique values in term:
   36 months: 988754
   60 months: 314853


In [10]:
# with engine.connect() as conn:
#     rows = conn.execute(text("SELECT target, COUNT(*) FROM loans_clean GROUP BY target")).fetchall()
#     print("Target distribution:")
#     for val, cnt in rows:
#         print(f"  {val}: {cnt}")


Target distribution:
  0: 1041952
  1: 261655


In [35]:
# import csv

# def run_feature_ingestion():
#     print("🚀 Step 1: Ingesting Raw Data with FICO Fix...")
#     # SQL Schema Update (Removed FICO fallback, strictly using raw data)
#     query_create = """
#     CREATE OR REPLACE TABLE loans_clean (
#         target INT, loan_amnt FLOAT, int_rate FLOAT, annual_inc FLOAT, dti FLOAT,
#         revol_util FLOAT, delinq_2yrs FLOAT, fico_score FLOAT,
#         grade VARCHAR(5), home_ownership VARCHAR(20), term VARCHAR(20)
#     );
#     """
#     with engine.connect() as conn:
#         conn.execution_options(isolation_level="AUTOCOMMIT").execute(text(query_create))

#     batch = []
#     with open(CSV_PATH, 'r', encoding='utf-8') as f:
#         reader = csv.DictReader(f)
#         for row in reader:
#             if row['loan_status'] not in ['Fully Paid', 'Charged Off']: continue
            
#             # --- PROBLEM SOLVING: FICO FIX ---
#             # Try original columns first, then 'last' columns
#             f_l = safe_float(row.get('fico_range_low')) or safe_float(row.get('last_fico_range_low'))
#             f_h = safe_float(row.get('fico_range_high')) or safe_float(row.get('last_fico_range_high'))
#             fico = (f_l + f_h) / 2 if (f_l and f_h) else None # Keep as None if missing

#             batch.append({
#                 'target': 1 if row['loan_status'] == 'Charged Off' else 0,
#                 'loan_amnt': safe_float(row.get('loan_amnt')),
#                 'int_rate': safe_float(row.get('int_rate')),
#                 'annual_inc': safe_float(row.get('annual_inc')),
#                 'dti': safe_float(row.get('dti')),
#                 'revol_util': safe_float(row.get('revol_util')),
#                 'delinq_2yrs': safe_float(row.get('delinq_2yrs')),
#                 'fico_score': fico, # SQL will handle NULL
#                 'grade': row.get('grade', 'U'),
#                 'home_ownership': row.get('home_ownership', 'OTHER'),
#                 'term': row.get('term', '36 months')
#             })
#             if len(batch) >= 10000:
#                 with engine.begin() as conn: conn.execute(loans_clean.insert(), batch)
#                 batch = []
#     if batch:
#         with engine.begin() as conn: conn.execute(loans_clean.insert(), batch)
#     print("✅ Ingestion with real FICO check complete.")

# run_feature_ingestion()

🚀 Step 1: Ingesting Raw Data with FICO Fix...
✅ Ingestion with real FICO check complete.


In [52]:
# def run_feature_engineering():
#     print("🚀 Step 2: Creating Behavioral Features...")
#     query = """
#     CREATE OR REPLACE TABLE loans_features AS
#     SELECT 
#         target,
#         -- 1. AFFORDABILITY Interaction
#         (loan_amnt / (annual_inc + 1)) AS payment_burden,
        
#         -- 2. CREDIT STRESS Interaction
#         (revol_util * dti) / 100.0 AS credit_stress_index,

#         -- 3. RAW SIGNALS
#         int_rate, dti, revol_util, annual_inc,
#         CASE WHEN home_ownership = 'RENT' THEN 1 ELSE 0 END AS is_rent,
#         CASE WHEN term LIKE '%60%' THEN 1 ELSE 0 END AS is_long_term
#     FROM loans_clean;
#     """
#     with engine.connect() as conn:
#         conn.execution_options(isolation_level="AUTOCOMMIT").execute(text(query))

In [53]:
# def run_preprocessing():
#     print("⚖️ Step 3: Scaling into Golden Matrix...")
#     query = """
#     CREATE OR REPLACE TABLE loans_processed AS
#     SELECT 
#         target, is_rent, is_long_term,
#         (fico_score - 300) / 550.0 AS fico_scaled,
#         (int_rate / 31.0) AS rate_scaled,
#         (dti / 100.0) AS dti_scaled,
#         (LOG(1 + annual_inc) / 17.0) AS income_scaled,
#         (payment_burden / 1.0) AS burden_scaled,
#         (risk_interaction / 50.0) AS risk_scaled
#     FROM loans_features;
#     """
#     with engine.connect() as conn:
#         conn.execution_options(isolation_level="AUTOCOMMIT").execute(text(query))
#     print("✅ Final scaling complete.")

# run_preprocessing()

⚖️ Step 3: Scaling into Golden Matrix...
✅ Final scaling complete.


In [63]:
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

# def train_and_evaluate():
#     print("\n🏋️ Step 3: Training with AUC-PR and Optimized Threshold...")
    
#     with engine.connect() as conn:
#         result = conn.execute(text("SELECT * FROM loans_processed"))
#         data = np.array(result.fetchall(), dtype=np.float32)

#     y, X = data[:, 0], data[:, 1:]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#     X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

#     # 1. Train specifically for Precision-Recall Area (aucpr)
#     model = xgb.XGBClassifier(
        
#         n_estimators=1000,
#         max_depth=3,              # Slightly shallower to prevent overfitting
#         learning_rate=0.05,
#         scale_pos_weight=4.0,     # Matches your 20/80 imbalance perfectly
#         colsample_bytree=0.2,     # Force the model to look at DTI and Income
#         subsample=0.8,            # Use 80% of data per tree
#         eval_metric='aucpr',
#         early_stopping_rounds=25
#     )
    
#     model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
#     # 2. Probability Generation
#     y_probs = model.predict_proba(X_test)[:, 1]

#     # 3. Finding the "Mathematical Sweet Spot" for Threshold
#     precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)
#     # F1 = 2 * (precision * recall) / (precision + recall)
#     f1_scores = (2 * precisions * recalls) / (precisions + recalls + 1e-9)
#     best_thresh = thresholds[np.argmax(f1_scores)]
    
#     # 4. Final Prediction with Optimized Threshold
#     y_pred = (y_probs >= best_thresh).astype(int)

#     print(f"🎯 Best F1 Threshold found: {best_thresh:.4f}")
#     print("\n📝 PERFORMANCE REPORT:")
#     print(classification_report(y_test, y_pred))
    
#     print("\n📉 CONFUSION MATRIX:")
#     print(confusion_matrix(y_test, y_pred))

# train_and_evaluate()


🏋️ Step 3: Training with AUC-PR and Optimized Threshold...
🎯 Best F1 Threshold found: 0.5214

📝 PERFORMANCE REPORT:
              precision    recall  f1-score   support

         0.0       0.88      0.66      0.75    208391
         1.0       0.31      0.63      0.42     52331

    accuracy                           0.65    260722
   macro avg       0.60      0.64      0.58    260722
weighted avg       0.76      0.65      0.68    260722


📉 CONFUSION MATRIX:
[[136578  71813]
 [ 19386  32945]]


In [40]:
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, f1_score

# def run_final_balanced_training():
#     print("\n🏋️ Step 5: Balancing Precision and Recall...")
    
#     # 1. Fetch data
#     with engine.connect() as conn:
#         result = conn.execute(text("SELECT * FROM loans_processed"))
#         data = np.array(result.fetchall(), dtype=np.float32)

#     y = data[:, 0]
#     X = data[:, 1:]
    
#     # 2. Drop 0-importance columns (term and fico) based on previous audit
#     X_cleaned = np.delete(X, [1, 2], axis=1) 

#     # 3. Split
#     X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, test_size=0.2, random_state=42, stratify=y)
#     X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

#     # 4. Train
#     model = xgb.XGBClassifier(
#         n_estimators=1000,
#         max_depth=5,
#         learning_rate=0.05,
#         scale_pos_weight=4, 
#         colsample_bytree=0.5,
#         eval_metric='aucpr',
#         early_stopping_rounds=20
#     )
    
#     model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

#     # --- THE FIX: Define y_probs clearly ---
#     # Probability for the 'Default' class (1)
#     y_probs = model.predict_proba(X_test)[:, 1] 

#     # 5. Threshold Optimization
#     precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)
#     f1_scores = (2 * precisions * recalls) / (precisions + recalls + 1e-9)
#     best_thresh = thresholds[np.argmax(f1_scores)]
    
#     # Apply
#     y_pred = (y_probs >= best_thresh).astype(int)

#     print(f"🎯 Mathematical Best Threshold: {best_thresh:.4f}")
#     print("\n📝 BALANCED PERFORMANCE REPORT:")
#     print(classification_report(y_test, y_pred))
    
#     print("\n📉 BALANCED CONFUSION MATRIX:")
#     print(confusion_matrix(y_test, y_pred))
    
#     # Return everything we need for the next cells
#     return model, X_test, y_test, y_probs

# # Execute the function
# model, X_test, y_test, y_probs = run_final_balanced_training()


🏋️ Step 5: Balancing Precision and Recall...
🎯 Mathematical Best Threshold: 0.5014

📝 BALANCED PERFORMANCE REPORT:
              precision    recall  f1-score   support

         0.0       0.88      0.61      0.72    208391
         1.0       0.30      0.67      0.42     52331

    accuracy                           0.63    260722
   macro avg       0.59      0.64      0.57    260722
weighted avg       0.76      0.63      0.66    260722


📉 BALANCED CONFUSION MATRIX:
[[128086  80305]
 [ 17410  34921]]


In [41]:
# print(f"Check: y_probs has {len(y_probs)} values.")

Check: y_probs has 260722 values.
